In [ ]:
# Устанавливаю необходимые библиотеки
from pymongo import MongoClient
from pprint import pprint   # Библиотека для удобного чтения файлов со сложной структурой
import json

In [ ]:
# Подключение к MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["alcomarket"]
products = db["products"]

In [ ]:
# 2. Очистка коллекции перед загрузкой (для повторного запуска)
products.drop()

In [ ]:
# 3. Загрузка данных из файла products.json
with open("products.json", "r") as f:
    data = json.load(f)
    products.insert_many(data)

In [ ]:
print("\n📦 Все товары:")
for doc in products.find():
    print (f'🍷')
    pprint(doc)

In [ ]:
print("\n🍷 Вина с рейтингом сомелье > 4.5:")
for doc in products.find({"type": "wine", "rating.sommelier": {"$gt": 4.7}}):
    pprint(doc)

In [ ]:
print("\n📑 Только имя и цена всех товаров:")
for doc in products.find({}, {"name": 1, "price": 1, "_id": 0}):
    pprint(doc)

In [ ]:
print("\n🌍 Уникальные страны происхождения:")
pprint(products.distinct("country"))

In [ ]:
print("\n📊 Средняя цена пива по странам:")
pipeline = [
    {"$match": {"type": "beer"}},
    {"$group": {"_id": "$country", "avgPrice": {"$avg": "$price"}}},
    {"$sort": {"avgPrice": -1}}
]
for doc in products.aggregate(pipeline):
    pprint(doc)

In [ ]:
print("\n🔄 Обновим цену Guinness до 2.99:")
products.update_one({"name": "Guinness Draught"}, {"$set": {"price": 2.99}})
pprint(products.find_one({"name": "Guinness Draught"}))

In [ ]:
print("\n➕ Добавим поле stock = 100 ко всем товарам:")
products.update_many({}, {"$set": {"stock": 100}})
pprint(products.find_one())

In [ ]:
print("\n❌ Удалим товары, которые недоступны (available: false):")
result = products.delete_many({"available": False})
print(f"Удалено документов: {result.deleted_count}")

In [ ]:
print("\n📦 Все товары после удаления:")
for doc in products.find():
    pprint(doc)

In [ ]:
print("\n📚 Индексы коллекции:")
pprint(products.index_information())

In [ ]:
# Создание индекса
products.create_index("type")

In [ ]:
print("\n🧾 Индексы после создания индекса по type:")
pprint(products.index_information())